# Handling Rotations

In [1]:
import numpy as np
np.set_printoptions(precision=4, suppress=True)

import sys
sys.path.append('../')

import pybvh
import matplotlib.pyplot as plt
from pathlib import Path

bvh_folder = Path('../bvh_data')

# Introduction — why do we care about rotations?

BVH files store joint orientations as **Euler angles**: three rotation values (in degrees) applied around specific axes in a specific order. Euler angles are compact and human-readable, but they have well-known drawbacks:

- **Gimbal lock** — when two axes align, one degree of freedom is lost, which can cause problems in interpolation and optimisation.
- **Discontinuities** — Euler angles can "jump" (e.g. from 179° to -179°), making them difficult for neural networks to learn.
- **Order-dependent** — the same three numbers produce different rotations depending on the axis order.

For these reasons, many applications convert Euler angles into other rotation representations before processing. The `pybvh.rotations` module provides conversions between the most commonly used representations:

| Representation | Shape | Typical use case |
|---|---|---|
| Euler angles | `(*, 3)` | BVH file storage, human inspection |
| Rotation matrices | `(*, 3, 3)` | Forward kinematics, composing rotations |
| 6D rotation | `(*, 6)` | Neural network training (continuous, no singularities) |
| Quaternions | `(*, 4)` | Smooth interpolation (SLERP), compact storage |
| Axis-angle | `(*, 3)` | SMPL/SMPL-X body models, pose estimation |

All conversions go through the **rotation matrix** as the central hub.

# Euler angles in BVH files

Let's first load a BVH file and look at how Euler angles are stored.

In [2]:
bvh = pybvh.read_bvh_file(bvh_folder / 'bvh_test1.bvh')
for node in bvh.nodes[:5]:
    if not node.is_end_site():
        print(f"{node.name:20s} rotation order: {node.rot_channels}")

Hips                 rotation order: ['Z', 'Y', 'X']
Spine                rotation order: ['Z', 'Y', 'X']
Spine1               rotation order: ['Z', 'Y', 'X']
Spine2               rotation order: ['Z', 'Y', 'X']
Spine3               rotation order: ['Z', 'Y', 'X']


Each joint has its own **rotation order**, stored in the `rot_channels` property. The order defines which elementary rotation is applied first, second, and third.

Motion data is stored as two structured arrays: `root_pos` (shape `(F, 3)` — root translation per frame) and `joint_angles` (shape `(F, J, 3)` — Euler angles in degrees per joint per frame).

In [5]:
print(f"root_pos shape:     {bvh.root_pos.shape}  (frames × 3)")
print(f"joint_angles shape: {bvh.joint_angles.shape}  (frames × joints × 3)")
print(f"\nFirst frame — root position: {bvh.root_pos[0]}")
print(f"First frame — first 3 joints (Euler angles in degrees):")
print(bvh.joint_angles[0, :3])

root_pos shape:     (56, 3)  (frames × 3)
joint_angles shape: (56, 24, 3)  (frames × joints × 3)

First frame — root position: [ 9.5267 -0.7495 36.2768]
First frame — first 3 joints (Euler angles in degrees):
[[88.2354 -1.0699  0.6448]
 [ 1.0586 -0.3574 -0.4139]
 [ 1.0857 -0.2637 -0.4131]]


## Why the rotation order matters

Euler rotations are **intrinsic** and applied by **pre-multiplication**: $R = R_1 \cdot R_2 \cdot R_3$, where $R_1$ corresponds to the first axis in the order. The same three angle values produce a completely different orientation if the order changes. Let's see this in action.

In [6]:
angles = np.array([30.0, 45.0, 60.0])  # same 3 numbers, two different orders

R_zyx = pybvh.rotations.euler_to_rotmat(angles, 'ZYX', degrees=True)
R_xyz = pybvh.rotations.euler_to_rotmat(angles, 'XYZ', degrees=True)

print("R with order ZYX:")
print(R_zyx)
print("\nR with order XYZ:")
print(R_xyz)

R with order ZYX:
[[ 0.6124  0.2803  0.7392]
 [ 0.3536  0.7392 -0.5732]
 [-0.7071  0.6124  0.3536]]

R with order XYZ:
[[ 0.3536 -0.6124  0.7071]
 [ 0.9268  0.1268 -0.3536]
 [ 0.1268  0.7803  0.6124]]


# Rotation matrices

A 3×3 rotation matrix $R$ is an orthogonal matrix with $\det(R) = +1$. It is the most general and unambiguous representation: any 3D rotation corresponds to exactly one 3x3 matrix (no singularities, no sign ambiguity).

## Euler → Rotation matrix

The function in `pybvh.rotations` that transform Euler angles into a Rotation Matrix is `euler_to_rotmat(angles, order, degrees)`.

For all functions, the `degrees` argument relate to the **Euler angles**. Here, if the input euler angles are in radians, then choose `degrees=False`.

In [7]:
angles_deg = np.array([30.0, 45.0, 60.0])
order = 'ZYX'

print(f"Euler angle: {str(angles_deg)} with order '{order}'.")

R = pybvh.rotations.euler_to_rotmat(angles_deg, order, degrees=True)
print("Rotation matrix:")
print(R)

Euler angle: [30. 45. 60.] with order 'ZYX'.
Rotation matrix:
[[ 0.6124  0.2803  0.7392]
 [ 0.3536  0.7392 -0.5732]
 [-0.7071  0.6124  0.3536]]


## Rotation matrix → Euler

We can also go back and recover the original angles. To do that, the function to use is `rotmat_to_euler(Rot_mat, order_wanted, degrees)`, with `order_wanted` the same order as original, here `'ZYX'`. 

You can also extract Euler angles in a *different* order — the angles will be different but will represent the same rotation. 

To obtain the Euler angles in radians, use `degrees=False`, otherwise use `degrees=True`.

In [8]:
recovered = pybvh.rotations.rotmat_to_euler(R, order, degrees=True)
print(f"Recovered angles (ZYX): {recovered}")

# Same rotation, but decomposed in a different Euler order
angles_xyz = pybvh.rotations.rotmat_to_euler(R, 'XYZ', degrees=True)
print(f"Same rotation as XYZ:   {angles_xyz}")

# Verify: both produce the same rotation matrix
R_check = pybvh.rotations.euler_to_rotmat(angles_xyz, 'XYZ', degrees=True)
print(f"Both give the same R?   {np.allclose(R, R_check)}")

Recovered angles (ZYX): [30. 45. 60.]
Same rotation as XYZ:   [ 58.3345  47.6632 -24.5972]
Both give the same R?   True


## Batch operations

All functions in `pybvh.rotations` accepts an array of angles as input. Here, if you input a 2D array of shape `(N, 3)`, you get `(N, 3, 3)` rotation matrices back.

*Hint*: the functions are batch-vectorised, which is much faster than looping. As much as possible, pass an array of angles to the function rather than manually looping through them.

In [9]:
batch_angles = np.array([
    [0, 0, 0],
    [30, 45, 60],
    [90, 0, 0],
    [-45, 90, 30],
], dtype=float)

batch_R = pybvh.rotations.euler_to_rotmat(batch_angles, 'ZYX', degrees=True)
print(f"Input shape:  {batch_angles.shape}")
print(f"Output shape: {batch_R.shape}")

Input shape:  (4, 3)
Output shape: (4, 3, 3)


# 6D rotation representation

The 6D representation was introduced by [Zhou et al. (CVPR 2019)](https://arxiv.org/abs/1812.07035) for neural network training. It concatenates the **first two columns** of the rotation matrix into a 6-element vector. Its key property is **continuity**: nearby rotations always map to nearby 6D vectors, unlike Euler angles or quaternions which can "wrap around".

## Euler → 6D

To convert Euler angles to 6D representation, use the function `euler_to_rot6d(angles, order, degrees)`.

In [10]:
angles_deg = np.array([30.0, 45.0, 60.0])
order = 'ZYX'

rot6d = pybvh.rotations.euler_to_rot6d(angles_deg, order, degrees=True)
print(f"6D vector (6 elements): {rot6d}")
print(f"Shape: {rot6d.shape}")

6D vector (6 elements): [ 0.6124  0.3536 -0.7071  0.2803  0.7392  0.6124]
Shape: (6,)


## 6D → Euler

For the inverse mapping, use the function `rot6d_to_euler(angles, order, degrees)`. Here again, `degrees` is how you want your output to be, in radians or in degrees.

In [13]:
recovered = pybvh.rotations.rot6d_to_euler(rot6d, order, degrees=True)
print(f"Original angles: {angles_deg}")
print(f"Recovered:       {recovered}")

Original angles: [30. 45. 60.]
Recovered:       [30. 45. 60.]


Another stenghth of this representation is that even a slightly noisy 6d-vector (e.g. from a neural network's output) is projected onto a valid rotation matrix.

# Quaternions

A unit quaternion $q = (w, x, y, z)$ with $\|q\| = 1$ provides a singularity-free representation of rotation in only 4 numbers. Quaternions are the default representation in many game engines and graphics libraries.

`pybvh` uses the **scalar-first** convention $(w, x, y, z)$ and enforces a **canonical form** where $w \geq 0$. This avoids the double-cover ambiguity where $q$ and $-q$ represent the same rotation.

## Euler → Quaternion


To convert Euler angles to quaternions, use the function `euler_to_quat(angles, order, degrees)`.

In [14]:
angles_deg = np.array([30.0, 45.0, 60.0])
order = 'ZYX'

q = pybvh.rotations.euler_to_quat(angles_deg, order, degrees=True)
print(f"Quaternion (w, x, y, z): {q}")

Quaternion (w, x, y, z): [0.8224 0.3604 0.4397 0.0223]


## Quaternion → Euler

To convert the other way, use the function `quat_to_euler(angles, order, degrees)`.

In [15]:
recovered = pybvh.rotations.quat_to_euler(q, order, degrees=True)
print(f"Original: {angles_deg}")
print(f"Recovered: {recovered}")

Original: [30. 45. 60.]
Recovered: [30. 45. 60.]


# Axis-angle

The axis-angle representation encodes a rotation as a single 3D vector $\mathbf{v}$ where:
- The **direction** $\hat{\mathbf{v}} = \mathbf{v} / \|\mathbf{v}\|$ is the rotation axis.
- The **magnitude** $\|\mathbf{v}\|$ is the rotation angle in radians.
- The **zero vector** $[0, 0, 0]$ represents the identity (no rotation).

This is the representation used by **SMPL** and **SMPL-X** body models, and by many pose estimation pipelines.

## Euler → Axis-angle

To convert Euler angles to axis-angle representation, use the function `euler_to_axisangle(angles, order, degrees)`.

In [16]:
angles_deg = np.array([30.0, 45.0, 60.0])
order = 'ZYX'

aa = pybvh.rotations.euler_to_axisangle(angles_deg, order, degrees=True)
angle_rad = np.linalg.norm(aa)
axis = aa / angle_rad

print(f"Axis-angle vector: {aa}")
print(f"Rotation axis:     {axis}")
print(f"Rotation angle:    {np.degrees(angle_rad):.2f}°")

Axis-angle vector: [0.7668 0.9354 0.0474]
Rotation axis:     [0.6335 0.7728 0.0391]
Rotation angle:    69.36°


## Axis-angle → Euler
To convert axis-angle to Euler angles, use the function `axisangle_to_euler(angles, order, degrees)`.

In [17]:
recovered = pybvh.rotations.axisangle_to_euler(aa, order, degrees=True)
print(f"Original:  {angles_deg}")
print(f"Recovered: {recovered}")

Original:  [30. 45. 60.]
Recovered: [30. 45. 60.]


# Working with Bvh objects

So far we used the low-level functions from `pybvh.rotations` directly on NumPy arrays. In practice, you will often want to convert an entire BVH animation at once. The `Bvh` class provides convenient methods that handle everything under the hood and return nicely organised arrays.

All `get_frames_as_*` methods return a tuple:
```
(root_pos, joint_data, joints)
```
where:
- `root_pos` — `(num_frames, 3)`: the root translation per frame.
- `joint_data` — `(num_frames, num_joints, *)`: the rotation data per joint per frame.
- `joints` — a list of joint node objects corresponding to the second axis of `joint_data`.

## Getting rotation matrices

In [18]:
bvh = pybvh.read_bvh_file(bvh_folder / 'bvh_test1.bvh')

root_pos, joint_rotmats, joints = bvh.get_frames_as_rotmat()
print(f"Number of joints:    {len(joints)}    -    Number of frames:    {root_pos.shape[0]}")
print(f"root_pos shape:      {root_pos.shape}")
print(f"joint_rotmats shape: {joint_rotmats.shape}")
print(f"First 5 joint names: {[j.name for j in joints[:5]]}")

Number of joints:    24    -    Number of frames:    56
root_pos shape:      (56, 3)
joint_rotmats shape: (56, 24, 3, 3)
First 5 joint names: ['Hips', 'Spine', 'Spine1', 'Spine2', 'Spine3']


## Getting other representations

In [19]:
root_pos, joint_6d, joints = bvh.get_frames_as_6d()
print(f"6D shape:          {joint_6d.shape}")

root_pos, joint_quats, joints = bvh.get_frames_as_quaternion()
print(f"Quaternion shape:  {joint_quats.shape}")

root_pos, joint_aa, joints = bvh.get_frames_as_axisangle()
print(f"Axis-angle shape:  {joint_aa.shape}")

6D shape:          (56, 24, 6)
Quaternion shape:  (56, 24, 4)
Axis-angle shape:  (56, 24, 3)


## Converting back

The `set_frames_from_*` methods let you write converted rotation data back into the Bvh object. Each joint's Euler order is automatically read from the joints attributes `rot_channels`. The following methods are available:
- `set_frames_from_6d`
- `set_frames_from_quaternion`
- `set_frames_from_axisangle`

Let's verify with one of them that a full round trip preserves the spatial coordinates of the skeleton.

In [20]:
# Get spatial coordinates BEFORE the conversion
spatial_before = bvh.get_spatial_coord()

# Convert to 6D and back
root_pos, joint_6d, joints = bvh.get_frames_as_6d()
bvh.set_frames_from_6d(root_pos, joint_6d)

# Get spatial coordinates AFTER the round trip
spatial_after = bvh.get_spatial_coord()

max_error = np.max(np.abs(spatial_before - spatial_after))
print(f"Max error in spatial coordinates after 6D round trip: {max_error:.2e}")
print(f"Positions preserved: {np.allclose(spatial_before, spatial_after, atol=1e-6)}")

Max error in spatial coordinates after 6D round trip: 7.11e-15
Positions preserved: True


# Changing Euler orders

Different BVH files (and even different joints within the same file) may use different Euler orders. A `bvh` object contains two methods to change Euler orders if needed:

- `single_joint_euler_angle(joint, new_order)` — change one joint's order.
- `change_all_euler_orders(new_order)` — change all joints to the same order.

Both methods convert through rotation matrices internally, so the physical rotations are preserved — only the Euler decomposition changes.

## Changing a single joint's order

In [21]:
bvh = pybvh.read_bvh_file(bvh_folder / 'bvh_test1.bvh')

# Look at the current order for the Hips joint
print(f"Hips order before: {bvh.root.rot_channels}")
print(f"Hips angles (frame 0): {bvh.joint_angles[0, 0]}")

# Change the Hips joint to XYZ order (not in-place, returns a new Bvh)
bvh_new = bvh.single_joint_euler_angle('Hips', 'XYZ', inplace=False)

print(f"\nHips order after:  {bvh_new.root.rot_channels}")
print(f"Hips angles (frame 0): {bvh_new.joint_angles[0, 0]}")

# The angles are different, but the rotation is the same

Hips order before: ['Z', 'Y', 'X']
Hips angles (frame 0): [88.2354 -1.0699  0.6448]

Hips order after:  ['X', 'Y', 'Z']
Hips angles (frame 0): [ 1.0892  0.6116 88.2356]


## Unifying all joints to one order

The `change_all_euler_orders` method is especially useful when a consistent structure across all joints is typically expected.

In [22]:
bvh = pybvh.read_bvh_file(bvh_folder / 'bvh_test1.bvh')


# Show orders before
print(f"Distinct orders before:")
for i, node in enumerate(bvh.nodes):
    if not node.is_end_site():
        print(f"{i:2d} {node.name:20s} rotation order: {node.rot_channels}")
    if i >= 3:
        break
print(f"... ")

# Convert everything to XYZ
bvh_unified = bvh.change_all_euler_orders('XYZ', inplace=False)

# Show orders after
print(f"\nDistinct orders after:")
for i, node in enumerate(bvh_unified.nodes):
    if not node.is_end_site():
        print(f"{i:2d} {node.name:20s} rotation order: {node.rot_channels}")
    if i >= 3:
        break
print(f"... ")

# Spatial coordinates preserved
spatial_orig = bvh.get_spatial_coord()
spatial_unified = bvh_unified.get_spatial_coord()
print(f"Same spatial coords?    {np.allclose(spatial_orig, spatial_unified, atol=1e-6)}")

Distinct orders before:
 0 Hips                 rotation order: ['Z', 'Y', 'X']
 1 Spine                rotation order: ['Z', 'Y', 'X']
 2 Spine1               rotation order: ['Z', 'Y', 'X']
 3 Spine2               rotation order: ['Z', 'Y', 'X']
... 

Distinct orders after:
 0 Hips                 rotation order: ['X', 'Y', 'Z']
 1 Spine                rotation order: ['X', 'Y', 'Z']
 2 Spine1               rotation order: ['X', 'Y', 'Z']
 3 Spine2               rotation order: ['X', 'Y', 'Z']
... 
Same spatial coords?    True


# Practical example: a typical ML preprocessing pipeline

Let's put it all together in a realistic scenario. Suppose you want to train a neural network on BVH motion data using the **6D rotation representation** (recommended for most deep learning applications). Here is a typical workflow:

In [23]:
joint_6d.ravel().shape

(8064,)

In [24]:
# 1. Load the BVH file
bvh = pybvh.read_bvh_file(bvh_folder / 'bvh_test1.bvh')

# 2. Extract 6D rotations + root positions
root_pos, joint_6d, joints = bvh.get_frames_as_6d()

print(f"Root positions:   {root_pos.shape}  — feed to the network as translation input")
print(f"6D rotations:     {joint_6d.shape} — feed to the network as rotation input")

# 3. You could flatten the 6D data to feed into a network, and then reshape it back after inference:
flat_6d = joint_6d.reshape(joint_6d.shape[0], -1)  # (frames, joints*6)
flat_6d = np.concatenate([root_pos, flat_6d], axis=1)  # (frames, 3 + joints*6)
print(f"Flattened 6D:     {flat_6d.shape}")

# 4. feed into the network ......
# ........
# ..... obtain new 6D data coming out of the network
predicted_6d = np.random.randn(*flat_6d.shape) #simulate network output with random data
new_root_pos = predicted_6d[:, :3]
new_joint_6d = predicted_6d[:, 3:].reshape(joint_6d.shape)

# 4. After network inference, reshape the output and set it back:
new_bvh = bvh.copy()
new_bvh.set_frames_from_6d(new_root_pos, new_joint_6d)

# 5. Save the result back as a BVH file
# new_bvh.to_bvh_file('output.bvh')
print("\n✓ Pipeline complete — BVH → 6D → (network) → 6D → BVH")

Root positions:   (56, 3)  — feed to the network as translation input
6D rotations:     (56, 24, 6) — feed to the network as rotation input
Flattened 6D:     (56, 147)

✓ Pipeline complete — BVH → 6D → (network) → 6D → BVH


# Summary of the conversion API

## Low-level functions (`pybvh.rotations`)

| Function | Input → Output |
|---|---|
| `euler_to_rotmat(angles, order, degrees)` | `(*, 3)` → `(*, 3, 3)` |
| `rotmat_to_euler(R, order, degrees)` | `(*, 3, 3)` → `(*, 3)` |
| `rotmat_to_rot6d(R)` | `(*, 3, 3)` → `(*, 6)` |
| `rot6d_to_rotmat(rot6d)` | `(*, 6)` → `(*, 3, 3)` |
| `rotmat_to_quat(R)` | `(*, 3, 3)` → `(*, 4)` |
| `quat_to_rotmat(q)` | `(*, 4)` → `(*, 3, 3)` |
| `rotmat_to_axisangle(R)` | `(*, 3, 3)` → `(*, 3)` |
| `axisangle_to_rotmat(aa)` | `(*, 3)` → `(*, 3, 3)` |

**Convenience wrappers** (go through rotmat internally): `euler_to_rot6d`, `rot6d_to_euler`, `euler_to_quat`, `quat_to_euler`, `euler_to_axisangle`, `axisangle_to_euler`.

## Bvh convenience methods

| Method | Description |
|---|---|
| `get_frames_as_rotmat()` | Euler → rotation matrices `(F, J, 3, 3)` |
| `get_frames_as_6d()` | Euler → 6D `(F, J, 6)` |
| `get_frames_as_quaternion()` | Euler → quaternions `(F, J, 4)` |
| `get_frames_as_axisangle()` | Euler → axis-angle `(F, J, 3)` |
| `set_frames_from_6d(root_pos, joint_rot6d)` | 6D → Euler (writes back into frames) |
| `set_frames_from_quaternion(root_pos, joint_quats)` | Quaternion → Euler |
| `set_frames_from_axisangle(root_pos, joint_aa)` | Axis-angle → Euler |
| `single_joint_euler_angle(name, order, inplace)` | Change one joint's Euler order |
| `change_all_euler_orders(order, inplace)` | Unify all joints to one Euler order |